In [11]:
import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark = SparkSession \
        .builder \
        .appName("Ejemplo de Spark") \
        .getOrCreate()

spark

In [8]:
df = spark.read.csv("../data/IBEX35")
df.show()

# Does not read headers

+------+----------+--------+---------+------+------------+--------+--------+--------+--------+
|   _c0|       _c1|     _c2|      _c3|   _c4|         _c5|     _c6|     _c7|     _c8|     _c9|
+------+----------+--------+---------+------+------------+--------+--------+--------+--------+
|Ticker|      Date|   Close|Reference|Volume|    Turnover|    Last|    High|     Low| Average|
|  AENA|02/01/2019|137.0000| 135.7500|143458| 19591073.50|137.0000|138.0000|133.9500|136.5631|
|  AENA|03/01/2019|137.0500| 137.0000|461151| 63059756.45|137.0500|139.1500|136.4000|137.4490|
|  AENA|04/01/2019|140.5000| 137.0500|110214| 15444845.65|140.5000|140.9500|137.7000|140.1351|
|  AENA|07/01/2019|139.1500| 140.5000| 92663| 12908476.55|139.1500|141.1000|138.2000|139.3056|
|  AENA|08/01/2019|140.0000| 139.1500|179857| 25206724.88|140.0000|141.0500|138.8000|140.1306|
|  AENA|09/01/2019|138.5000| 140.0000|163959| 22859110.75|138.5000|141.6500|138.3500|138.8855|
|  AENA|10/01/2019|139.2500| 138.5000|181664| 2520

In [12]:
df = spark.read.option("header", "true").csv("../data/IBEX35")
df.show()

+------+----------+--------+---------+------+------------+--------+--------+--------+--------+
|Ticker|      Date|   Close|Reference|Volume|    Turnover|    Last|    High|     Low| Average|
+------+----------+--------+---------+------+------------+--------+--------+--------+--------+
|  AENA|02/01/2019|137.0000| 135.7500|143458| 19591073.50|137.0000|138.0000|133.9500|136.5631|
|  AENA|03/01/2019|137.0500| 137.0000|461151| 63059756.45|137.0500|139.1500|136.4000|137.4490|
|  AENA|04/01/2019|140.5000| 137.0500|110214| 15444845.65|140.5000|140.9500|137.7000|140.1351|
|  AENA|07/01/2019|139.1500| 140.5000| 92663| 12908476.55|139.1500|141.1000|138.2000|139.3056|
|  AENA|08/01/2019|140.0000| 139.1500|179857| 25206724.88|140.0000|141.0500|138.8000|140.1306|
|  AENA|09/01/2019|138.5000| 140.0000|163959| 22859110.75|138.5000|141.6500|138.3500|138.8855|
|  AENA|10/01/2019|139.2500| 138.5000|181664| 25208702.51|139.2500|139.5000|137.5500|138.8824|
|  AENA|11/01/2019|139.4500| 139.2500|710391| 9912